In [20]:
import requests
from pydantic import BaseModel
from datetime import datetime

In [21]:
# Define the Pydantic input data model
class InputData(BaseModel):
    fixed_acidity: float
    volatile_acidity: float
    citric_acid: float
    residual_sugar: float
    chlorides: float
    free_sulfur_dioxide: float
    total_sulfur_dioxide: float
    density: float
    pH: float
    sulphates: float
    alcohol: float


In [22]:
# Create sample input data
sample_data = {
    "fixed_acidity": 7.4,
    "volatile_acidity": 0.7,
    "citric_acid": 0.0,
    "residual_sugar": 1.9,
    "chlorides": 0.076,
    "free_sulfur_dioxide": 11.0,
    "total_sulfur_dioxide": 34.0,
    "density": 0.9978,
    "pH": 3.51,
    "sulphates": 0.56,
    "alcohol": 9.4
}

In [23]:
# Instantiate the input data model
input_data = InputData(**sample_data)

endpoint_url = "http://localhost:8000/predict/"

response = requests.post(endpoint_url, json=input_data.dict())
if response.status_code == 200:
    prediction = response.json()["prediction"]
    print("Prediction:", prediction)
else:
    print("Error:", response.text)

Prediction: 5.01


In [24]:
# Send a POST request to save the prediction
save_prediction_endpoint = "http://localhost:8000/save_prediction/"
response = requests.post(save_prediction_endpoint, json={
    "data": sample_data,
    "prediction": {"prediction": prediction}
})

if response.status_code == 200:
    print("Prediction saved successfully.")
else:
    print("Error saving prediction:", response.text)

Prediction saved successfully.


In [26]:
# Test get past predictions
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 31)

# Define the URL for the endpoint
endpoint_url = "http://localhost:8000/get_past_predictions/"

# Send a GET request to retrieve past predictions
response = requests.get(endpoint_url, params={"start_date": start_date, "end_date": end_date})

if response.status_code == 200:
    predictions = response.json()
    print("Past Predictions:")
    for prediction in predictions:
        print(prediction)
else:
    print("Error:", response.text)

Past Predictions:
{'prediction': 5.01}
{'prediction': 5.01}
{'prediction': 5.01}
{'prediction': 5.01}
